__Name:__ Daniel Vermaas<br>__UvA_ID:__ 12208698

In [4]:
import requests
from collections import Counter, defaultdict
import numpy as np
import math

In this homework, we will compress Hamlet using arithmetic coding.

In [5]:
HAMLET = requests.get(
    "https://github.com/amsqi/iit21-homework/raw/main/material/hamlet.txt"
).content.decode("ascii", errors="ignore")
len(HAMLET)

179096

Like last week, we use the following conventions:
- **probability distributions** are represented by dictionaries that map symbols to probabilities, i.e., the probability $P(x)$ of outcome $x$ corresponds to the dictionary entry `P[x]`
- **bitstrings** are represented by Python lists or tuples that contain 0s and 1s (for simplicity)

# 1. Language Model

Last week, you implemented a function `empirical` that computed the empirical probability distribution of letters in a given string. Your solution probably looked similar to the following function:

In [6]:
def empirical(s):
    c = Counter(s)
    N = len(s)
    return {x: n / N for x, n in c.items()}


empirical("Hello")

{'H': 0.2, 'e': 0.2, 'l': 0.4, 'o': 0.2}

Compression is intimely connected with modeling the source that we want to compress. We know this very clearly from Shannon's theorem as well as from Huffman coding, which each relied on a probability distribution P modeling an IID data source. Sometimes this also happens implicitly, like in the LZ algorithm (where we can think of the phrases that are being assembled as a "model" of the data).

Arithmetic coding also relies on an explicit probabilistic model of the data. This week, we will consider so-called **digram models** (also known as bigram models). A diagram model is given by a **conditional probability distributions** $P(y|x)$, which models the probability that the next letter is $y$ if the previous letter was $x$.
We will represent conditional probability distributions by Python dictionaries that map symbols to probability distributions, so that the conditional probability $P(y|x)$ is corresponds to the dictionary entry `P[x][y]`. Here, we also allow `x` to be `None` and interpret `P[None][y]` as the probability distribution of the first letter of the string.

We can  we can use the `empirical` function from above to construct a very simple digram model, where `P[x][y]` equals by the frequency of `y` in the given string, so does not depend on `x` at all. In fact, this should better be called a "unigram" model, so that is the name that we use for the function:

In [7]:
def unigram(s):
    # compute frequencies of single letter
    Q = empirical(s)

    # use as P[x] for all letters, including the first (x = None)
    P = {}
    P[None] = Q
    for x in s:
        P[x] = Q
    return P


unigram("Hello")

{'H': {'H': 0.2, 'e': 0.2, 'l': 0.4, 'o': 0.2},
 None: {'H': 0.2, 'e': 0.2, 'l': 0.4, 'o': 0.2},
 'e': {'H': 0.2, 'e': 0.2, 'l': 0.4, 'o': 0.2},
 'l': {'H': 0.2, 'e': 0.2, 'l': 0.4, 'o': 0.2},
 'o': {'H': 0.2, 'e': 0.2, 'l': 0.4, 'o': 0.2}}

Clearly this model is silly, since it does not model any correlations between subsequent letters. **To do better, implement a function that computes the empirical conditional probability of subsequent letters in a given string.** That is, your function should compute
$$P(y|x) = \frac {N_{x,y}} {\sum_{y'} N_{x,y'}},$$
where $N_{x,y}$ denotes the number of times that `xy` appears as a substring of the given string.
Please return the conditional distribution in the format described above, i.e., `P[x][y]` should correspond to $P(y|x)$.
We already gave you a head start and took care of `P[None]` by setting it to "deterministic" probability distribution corresponding to the first letter of the string.

In [8]:
def digram(s):
    P = {}

    # the first letter is deterministic
    P[None] = {s[0]: 1}

    # TODO: can you implement the rest?
    for i in range(1, len(s)):
      if s[i-1] not in P:
        P[s[i-1]] = ""
      P[s[i-1]] += s[i]
    return {key : value if key is None else empirical(value) for key, value in P.items()}

digram("Abracadabra")

{'A': {'b': 1.0},
 None: {'A': 1},
 'a': {'b': 0.3333333333333333,
  'c': 0.3333333333333333,
  'd': 0.3333333333333333},
 'b': {'r': 1.0},
 'c': {'a': 1.0},
 'd': {'a': 1.0},
 'r': {'a': 1.0}}

The following code tests whether your function works correctly:

In [9]:
P = digram("Abracadabra")

# the first letter is always A
assert P[None] == {"A": 1.0}

# after A always follows b
assert P["A"] == {"b": 1.0}

# after b always comes r
assert P["b"] == {"r": 1.0}

# after r, c, d always comes a
assert P["r"] == {"a": 1.0} and P["c"] == {"a": 1.0} and P["d"] == {"a": 1.0}

# after r comes either of {b, c, d}, with equal probabilities
assert P["a"] == {"b": 1 / 3, "c": 1 / 3, "d": 1 / 3}

# 2. Helpers

We need two ingredients to build the arithmetic coding algorithm.
The first is the ability to compute binary expansions of numbers $0 \leq f < 1$
Let $f=0.b_1b_2b_3\dots$ be the standard binary expansion as discussed in class.
**The following function should return the first `l` bits in the binary expansion of `f`. Can you implement it?**

In [10]:
def binary_expansion(f, l):
    # TODO: can you implement this?
    out = []
    for i in range(l):
      f *= 2
      out.append(int(f//1))
      f %= 1
    return out

binary_expansion(1 / 3, 10)

[0, 1, 0, 1, 0, 1, 0, 1, 0, 1]

Here are some tests to make sure that your code works fine:

In [11]:
assert binary_expansion(1 / 3, 2) == [0, 1]
assert binary_expansion(1 / 3, 10) == [0, 1, 0, 1, 0, 1, 0, 1, 0, 1]
assert binary_expansion(5 / 6, 2) == [1, 1]
assert binary_expansion(5 / 6, 10) == [1, 1, 0, 1, 0, 1, 0, 1, 0, 1]

The second ingredient that we will need is the ability to compute lower and upper cumulative probabilities.
We have implemented the first for you:

In [12]:
def lower_cumulative(P):
    symbols = sorted(P)
    Q = {x: sum(P[y] for y in symbols[:k]) for k, x in enumerate(symbols)}
    return Q


P = {"A": 2 / 3, "B": 1 / 6, "C": 1 / 6}
assert lower_cumulative(P) == {"A": 0, "B": 2 / 3, "C": 2 / 3 + 1 / 6}

**Can you implement the following function to compute upper cumulative probabilities?**

In [13]:
def upper_cumulative(P):
    # TODO: can you implement this?
    symbols = sorted(P)
    return {x: sum(P[y] for y in symbols[:k+1]) for k, x in enumerate(symbols)}

upper_cumulative({"A": 2 / 3, "B": 1 / 6, "C": 1 / 6})

{'A': 0.6666666666666666, 'B': 0.8333333333333333, 'C': 0.9999999999999999}

Here is a test to make sure everything is in good order:

In [14]:
P = {"A": 2 / 3, "B": 1 / 6, "C": 1 / 6}
assert upper_cumulative(P) == {
    "A": 2 / 3,
    "B": 2 / 3 + 1 / 6,
    "C": 2 / 3 + 1 / 6 + 1 / 6,
}

# 3. Arithmetic Coding

Now it's time to assemble all ingredients.
**Your final task is to implement the arithmetic coding algorithm as discussed in class.** Your function should take as input a string of symbols (`s`) and a conditional probability distribution (`P`) corresponding the digram model, and it should return as output a bitstring.

In [15]:
def arithmetic_encode(s, P):
    # TODO: can you implement this
    q = 0
    r = p = 1
    output = []
    prev_letter = None
    for letter in s:
      # step 1
      r = q + p * upper_cumulative(P[prev_letter])[letter]
      q = q + p * lower_cumulative(P[prev_letter])[letter]
      #print('step 1 (lower, upper, q,r)', lower_cumulative(P[prev_letter])[letter], upper_cumulative(P[prev_letter])[letter], q, r)
      prev_letter = letter
      # step 2
      while r <= 1/2 or q >= 1/2:
        if r <= 1/2:
          b = 0
        else:
          b = 1
        output.append(b)
        #print('s2out', b)
        r = (2 * r) - b
        q = (2 * q) - b
      #print('step 2 (q,r)', q, r)
      # step 3
      p = r - q
      #print('step 3 (p)', p)
    return output + binary_expansion((q+r)/2, math.ceil(np.log2(2/p)))
      
arithmetic_encode("ABA", unigram("ABBB"))

[0, 0, 0, 1, 0, 1]

In [16]:
TEXT = "Hello"
print(arithmetic_encode(TEXT, digram(TEXT)))

[0, 1, 1]


Here are some tests of increasing difficulty:

In [17]:
# this is the probability distribution discussed in class (P(A) = 2/3, P(B) = 1/3)
P = unigram("AAB")

# for a single letter, arithmetic coding is the same as the Shannon-Fano-Elias code, so we can compare with the example discussed in class
assert arithmetic_encode("A", P) == [0, 1]
assert arithmetic_encode("B", P) == [1, 1, 0]

# now compress a string of length 300 (note that 277/300 is already close to the entropy)
assert len(arithmetic_encode("AAB" * 100, P)) == 277

In [18]:
TEXT = "Hello"
assert arithmetic_encode(TEXT, unigram(TEXT)) == [0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1]
assert arithmetic_encode(TEXT, digram(TEXT)) == [0, 1, 1]

Finally, let us see how well we can compress Hamlet using either the language model.
For the unigram model, your code should compare similarly to last week and reach about 58%.
For the digram model, arithmetic coding should do markedly better -- does this match your output?

*Hint: If your code runs too slow, try to avoid re-computing the cumultative probabilities from scratch for every input letter!*

In [19]:
# 2m 59s runtime, not too bad.
models = {"unigram": unigram, "digram": digram}

for name, model in models.items():
    compressed = arithmetic_encode(HAMLET, model(HAMLET))
    compressed_bytes = len(compressed) / 8
    R = compressed_bytes / len(HAMLET)
    print(f"Compression rate using {name} model: {R:2.0%}")

Compression rate using unigram model: 58%
Compression rate using digram model: 45%


**Challenge problems (purely optional):**
* Program a decompressor! The teaching assistants will explain to you how to do so in the tutorials...
* Turn your code into a "true" streaming algorithm. That is, make sure that your algorithm still works if the input parameter is an iterable rather than a string of symbols.
* Implement a trigraph model and see if it performs even better.
* In fact, why don't you allow for language models that arbitrarily depend on preceding characters. It may be useful to introduce a `LanguageModel` base class with an `update(x)` function that is called upon reading a letter `x`, as well as functions `P(y)` and `Q(y)` that compute the lower and upper conditional cumulative probability of `y`.
* For a true compressor, you would also need to store the probability distribution as part of the compressed data. An alternative is to use a language model that learns the language "on the fly". One way to learn the unigram distribution is *Laplace's rule*, which you discussed in the exercise class. Implement it!

In [20]:
def bitlist_to_float(bitlist):
  out = 0
  for i in range(len(bitlist)):
    if bitlist[i] == 1:
      out += 1/(2**(i+1))
  return out

In [21]:
def arithmetic_decode(s, P, N):
  b = bitlist_to_float(s)
  q = 0
  p = 1
  x = ""
  prev_letter = None
  for i in range(N):
    for letter in P[prev_letter]:
      dq = q + p * lower_cumulative(P[prev_letter])[letter]
      dr = q + p * upper_cumulative(P[prev_letter])[letter]
      if dq <= b and b < dr:
        #print('t1', dq, dr, lower_cumulative(P[prev_letter])[letter], upper_cumulative(P[prev_letter])[letter])
        x += letter
        q = dq
        p = dr - dq
        #print('t2', p, dr, dq)
        break
    prev_letter = x[-1]
  return x

  
print(arithmetic_decode(arithmetic_encode("A", unigram("AAB")), unigram("AAB"), 1))
print(arithmetic_decode(arithmetic_encode("B", unigram("AAB")), unigram("AAB"), 1))
print(arithmetic_decode(arithmetic_encode("ABA", unigram("AAB")), unigram("AAB"), 3))
print(arithmetic_decode(arithmetic_encode("ACCDA", digram("ACBCCABCCDAC")), digram("ACBCCABCCDAC"), 5))

A
B
ABA
ACCDA


__1a:__<br>


In [22]:
P = {None : {"A" : 5/6, "H" : 1/6}, "A" : {"i" : 2/3, "y" : 1/3}, "H" : {"i" : 1/5, "y" : 4/5}}
print('encoding \'Hi\':', arithmetic_encode("Hi", P))

encoding 'Hi': [1, 1, 0, 1, 1, 0]


__1b:__<br>

Lets see if we can find the '10' decoding by encoding all posibilites

In [23]:
print('encoding \'Ai\':', arithmetic_encode("Ai", P))
print('encoding \'Ay\':', arithmetic_encode("Ay", P))
print('encoding \'Hy\':', arithmetic_encode("Hy", P))

encoding 'Ai': [0, 1]
encoding 'Ay': [1, 0, 1]
encoding 'Hy': [1, 1, 1, 0]


We cannot seem to find '10' bitstring, lets use the decoder:

In [24]:
print('decoding \'10\':', arithmetic_decode([1, 0], P, 2))

decoding '10': Ai
